In [1]:
from fastbook import *
from fastai.vision.all import *
from fastai.callback.fp16 import *
path = untar_data(URLs.PETS)
Path.BASE_PATH = path


In [2]:
pets = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(seed=42),
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = pets.dataloaders(path/"images")

/home/miles/anaconda3/envs/fastai/lib/python3.8/site-packages/torch/_tensor.py:1051: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:766.)
  ret = func(*args, **kwargs)


In [29]:
learn = cnn_learner(dls, resnet34, metrics=error_rate) # .to_fp16() # half-floats. this would make it faster but fail to pickle for some reason..

In [30]:
learn.fine_tune(6, freeze_epochs=3)

epoch,train_loss,valid_loss,error_rate,time
0,2.623831,0.559401,0.177943,00:34
1,0.955349,0.283005,0.096752,00:34
2,0.530305,0.243878,0.082544,00:34


epoch,train_loss,valid_loss,error_rate,time
0,0.345994,0.238715,0.072395,00:43
1,0.338326,0.346700,0.114344,00:43
2,0.269034,0.253178,0.075101,00:43
3,0.189486,0.219765,0.067659,00:44
4,0.125150,0.195959,0.061570,00:45
5,0.089497,0.192652,0.055480,00:45


In [31]:
input_img = Path("/home/miles/served_files/moof.jpg") # path/"images/beagle_186.jpg"
p, pn, cats = learn.predict(input_img)
print(p, cats[pn])

Persian TensorBase(0.7805)


In [48]:
%%writefile pet_classification_service.py

from bentoml.frameworks.fastai import FastaiModelArtifact
from bentoml.adapters import FileInput
from fastcore.utils import tuplify, detuplify

import bentoml
import torchvision # not imported by default, to help with pickling
# import datablock_utils

# img conversion
from fastai.vision.core import PILImage

@bentoml.artifacts([FastaiModelArtifact('learner')])
@bentoml.env(infer_pip_packages=True)
class PetClassificationService(bentoml.BentoService):
    @bentoml.api(input=FileInput(), batch=True)
    def predict(self, files):
        # TODO: learner.get_preds
        results = [self.artifacts.learner.predict(PILImage.create(i)) for i in files]
        return results

Overwriting pet_classification_service.py


In [49]:
import sys
import os
sys.path.append(os.getcwd())

import bentoml

# # from fastcore.utils import remove_patches_path
from pet_classification_service import PetClassificationService

# learn.metrics = []

svc = PetClassificationService()
svc.pack('learner', learn)

# Fastcore modify Pathlib module from python, need to remove the patch in order to save
# with remove_patches_path():
saved_path = svc.save()

# bentoml serve PetClassification:latest     

[2021-12-30 01:12:22,789] WARNING - BentoML by default does not include spacy and torchvision package when using FastaiModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService
[2021-12-30 01:12:24,781] INFO - BentoService bundle 'PetClassificationService:20211230011222_E1C7CD' saved to: /home/miles/bentoml/repository/PetClassificationService/20211230011222_E1C7CD
